In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit,col,explode
import pyspark.sql.functions as f 


In [2]:
# Extract

spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()


In [3]:
spark

In [5]:
df = spark.read.text("data/WordData1.txt")
df.show(truncate=False)

+-----------------------------------------------------------------------------------------+
|value                                                                                    |
+-----------------------------------------------------------------------------------------+
|This is a Japanese doll                                                                  |
|The team members were hard to tell apart since they all wore their hair in a ponytail    |
|As the years pass by we all know owners look more and more like their dogs               |
|If you don't like toenails you probably shouldn't look at your feet                      |
|He was disappointed when he found the beach to be so sandy and the sun so sunny          |
|When he encountered maize for the first time he thought it incredibly corny              |
|Situps are a terrible way to end your day                                                |
|Toddlers feeding raccoons surprised even the seasoned park ranger              

In [17]:
df.withColumn("splitdata",f.split("value"," "))\
    .withColumn("words",f.explode("splitdata"))\
    .select("words").groupBy("words").count()\
    .alias("count").orderBy("count",ascending=False).show()

+-----+-----+
|words|count|
+-----+-----+
|   to|   88|
|    a|   76|
|  the|   76|
|  was|   44|
|   he|   40|
|   is|   40|
|   it|   36|
|  The|   32|
|   He|   28|
| that|   28|
|  and|   28|
|  her|   24|
|  you|   24|
|    I|   24|
|   of|   24|
|   so|   24|
|   in|   24|
|   be|   24|
|   if|   20|
|  are|   16|
+-----+-----+
only showing top 20 rows



In [9]:
# tranformatin 
df2 = df.withColumn("splitdata",f.split("value"," "))
df2.withColumn("words",f.explode("splitdata")).show()

+--------------------+--------------------+--------+
|               value|           splitdata|   words|
+--------------------+--------------------+--------+
|This is a Japanes...|[This, is, a, Jap...|    This|
|This is a Japanes...|[This, is, a, Jap...|      is|
|This is a Japanes...|[This, is, a, Jap...|       a|
|This is a Japanes...|[This, is, a, Jap...|Japanese|
|This is a Japanes...|[This, is, a, Jap...|    doll|
|The team members ...|[The, team, membe...|     The|
|The team members ...|[The, team, membe...|    team|
|The team members ...|[The, team, membe...| members|
|The team members ...|[The, team, membe...|    were|
|The team members ...|[The, team, membe...|    hard|
|The team members ...|[The, team, membe...|      to|
|The team members ...|[The, team, membe...|    tell|
|The team members ...|[The, team, membe...|   apart|
|The team members ...|[The, team, membe...|   since|
|The team members ...|[The, team, membe...|    they|
|The team members ...|[The, team, membe...|   

In [7]:
# transformation
df2 = df.withColumn('splitedData',f.split('value',' '))
df3 = df2.withColumn('words',explode('splitedData'))
word_df = df3.select('words')
word_df.show()

+--------+
|   words|
+--------+
|    This|
|      is|
|       a|
|Japanese|
|    doll|
|     The|
|    team|
| members|
|    were|
|    hard|
|      to|
|    tell|
|   apart|
|   since|
|    they|
|     all|
|    wore|
|   their|
|    hair|
|      in|
+--------+
only showing top 20 rows



In [8]:
word_count = word_df.groupBy('words').count()

In [5]:
b=0.1+0.2
c=0.3
c==b

False

In [1]:
print(0.1+0.2==0.3)

False


In [9]:
word_count.show()

+-----------+-----+
|      words|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows



In [19]:
# LOAD 
driver = "com.mysql.jdbc.Driver"
url = 'jdbc:mysql://database-2.cjbysn8oqrq1.us-east-1.rds.amazonaws.com/'
table = 'avinash_schema.wordcount'
user='admin'
password='ad789min'

word_count.write.mode('append').format("jdbc").option("url",url).option("driver",driver)\
    .option("dbtable",table).option("mode","append").option("user",user)\
        .option("password",password).save()

In [18]:
# LOAD 
driver1 = "com.mysql.jdbc.Driver"
url1 = 'jdbc:mysql://localhost:3306/'
table1 = 'study.wordcount'
user1='root'
password1='root'

word_count.write.mode('append').format('jdbc').option("url",url1)\
    .option("driver",driver1)\
    .option('dbtable',table1).option('mode','append').option('user',user1)\
        .option('password',password1).save()

In [18]:
# from pyspark.sql import SparkSession

# spark = SparkSession\
#     .builder\
#     .appName("Word Count")\
#     .getOrCreate()

dataframe_mysql = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/exam")\
    .option("driver", "com.mysql.cj.jdbc.Driver")\
    .option("dbtable", "books").option("user", "root")\
    .option("password", "root").load()

print(dataframe_mysql.columns)

['book_id', 'title', 'price']


In [17]:
dataframe_mysql.show()

+-------+--------------------+-----+
|book_id|               title|price|
+-------+--------------------+-----+
|  12345|  Python Programming|   29|
|  12346|         Learn MySQL|   23|
|  12347|Data Science Cook...|   27|
|  12367|           My dreams|   50|
|  13456|       Eat that frog|   34|
+-------+--------------------+-----+



In [18]:
def fun(num):
    if num<1:
        return 0
    elif num%2==0:
        return fun(num-1)
    else:
        return num+fun(num-2)

fun(8)

16

In [19]:

# Python3 program for the above approach
def max_profit(prices: list, days: int) -> int:
  
    profit = 0
  
    for i in range(1, days):
  
        # checks if elements are adjacent and in increasing order
        if prices[i] > prices[i-1]:
  
            # difference added to 'profit'
            profit += prices[i] - prices[i-1]
  
    return profit

def StockPicker(arr):
    min_price = float('inf')
    max_profit = -1
    for price in arr:
        min_price = min(min_price, price)
        max_profit = max(max_profit, price - min_price)
    return max_profit if max_profit > 0 else -1


In [20]:
in12 = [10,12,4,5,9]


In [22]:
max_profit(in12,len(in12))

7

In [24]:
StockPicker(in12)

5